Based on [this DiCE example notebook](https://github.com/interpretml/DiCE/blob/78ca5391467ba12e38730e71577fbe421d9f0ba2/docs/source/notebooks/DiCE_with_advanced_options.ipynb).

In [1]:
# import DiCE
import dice_ml
from dice_ml.utils import helpers # helper functions

# Tensorflow libraries
import tensorflow as tf
from tensorflow import keras

In [2]:
dataset = helpers.load_adult_income_dataset()

In [3]:
dataset.head()

,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,39,Government,Bachelors,Single,White-Collar,White,Male,40,0
1,50,Self-Employed,Bachelors,Married,White-Collar,White,Male,13,0
2,38,Private,HS-grad,Divorced,Blue-Collar,White,Male,40,0
3,53,Private,School,Married,Blue-Collar,Other,Male,40,0
4,28,Private,Bachelors,Married,Professional,Other,Female,40,0


In [4]:
d = dice_ml.Data(dataframe=dataset, continuous_features=['age', 'hours_per_week'], outcome_name='income')

In [5]:
# seeding random numbers for reproducability
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

In [6]:
train, _ = d.split_data(d.normalize_data(d.one_hot_encoded_data))
X_train = train.loc[:, train.columns != 'income']
y_train = train.loc[:, train.columns == 'income']

ann_model = keras.Sequential()
ann_model.add(keras.layers.Dense(20, input_shape=(X_train.shape[1],), kernel_regularizer=keras.regularizers.l1(0.001), activation=tf.nn.relu))
ann_model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

ann_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.01), metrics=['accuracy'])
ann_model.fit(X_train, y_train, validation_split=0.20, epochs=100, verbose=0, class_weight={0:1,1:2})
# the training will take some time for 100 epochs.
# you can wait or set verbose=1 to see the progress of training.

In [7]:
backend = 'TF'+tf.__version__[0] # TF2
m = dice_ml.Model(model=ann_model, backend=backend)

In [8]:
# initiate DiCE
exp = dice_ml.Dice(d, m)

In [9]:
# query instance in the form of a dictionary; keys: feature name, values: feature value
query_instance = {'age':22, 
                  'workclass':'Private', 
                  'education':'HS-grad', 
                  'marital_status':'Single', 
                  'occupation':'Service',
                  'race': 'White', 
                  'gender':'Female', 
                  'hours_per_week': 45}

In [10]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(query_instance, total_CFs=4, desired_class="opposite")

Diverse Counterfactuals found! total time taken: 00 min 32 sec


In [11]:
# visualize the resutls
dice_exp.visualize_as_dataframe()

Query instance (original outcome : 0)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,22.0,Private,HS-grad,Single,Service,White,Female,45.0,0.009411



Diverse Counterfactual set (new outcome : 1)


,age,workclass,education,marital_status,occupation,race,gender,hours_per_week,income
0,57.0,Private,Doctorate,Single,White-Collar,White,Female,45.0,0.724
1,36.0,Private,Prof-school,Married,Service,White,Female,37.0,0.869
2,22.0,Self-Employed,Doctorate,Married,Service,White,Female,45.0,0.755
3,43.0,Private,HS-grad,Married,White-Collar,White,Female,63.0,0.822
